In [3]:
from parser import parse_files
from createPopulation import createPopulation
from makespan import makespan
from avaliarPop import avaliarPop
from retornaMelhorSolucao import retornaMelhorSolucao
from selecionarPop import selecionarPop
from recombinacao import recombinacao
from mutacao import mutacao
from selecionarNovaGeracao import selecionarNovaGeracao


import time

In [ ]:
POPULATION_SIZE = 100
TIME_LIMIT = 1000

### Ler e parsear os dados

In [52]:
files_path = ['files/tai20_10.txt', 'files/tai20_10.txt', 'files/tai20_20.txt', 'files/tai50_5.txt', 'files/tai50_10.txt', 'files/tai50_20.txt', 'files/tai100_5.txt', 'files/tai100_10.txt', 'files/tai100_20.txt', 'files/tai200_10.txt']

In [53]:
instances_list = parse_files(files_path)
# o campo processing possui as instancias a serem calculadas
print(instances_list[0])

{'processing': [[74, 21, 58, 4, 21, 28, 58, 83, 31, 61, 94, 44, 97, 94, 66, 6, 37, 22, 99, 83], [28, 3, 27, 61, 34, 76, 64, 87, 54, 98, 76, 41, 70, 43, 42, 79, 88, 15, 49, 72], [89, 52, 56, 13, 7, 32, 32, 98, 46, 60, 23, 87, 7, 36, 26, 85, 7, 34, 36, 48], [60, 88, 26, 58, 76, 98, 29, 47, 79, 26, 19, 48, 95, 78, 77, 90, 24, 10, 85, 55], [54, 66, 12, 57, 70, 82, 99, 84, 16, 41, 23, 11, 68, 58, 30, 5, 5, 39, 58, 31], [92, 11, 54, 97, 57, 53, 65, 77, 51, 36, 53, 19, 54, 86, 40, 56, 79, 74, 24, 3], [9, 8, 88, 72, 27, 22, 50, 2, 49, 82, 93, 96, 43, 13, 60, 11, 37, 91, 84, 67], [4, 18, 25, 28, 95, 51, 84, 18, 6, 90, 69, 61, 57, 5, 75, 4, 38, 28, 4, 80], [25, 15, 91, 49, 56, 10, 62, 70, 76, 99, 58, 83, 84, 64, 74, 14, 18, 48, 96, 86], [15, 84, 8, 30, 95, 79, 9, 91, 76, 26, 42, 66, 70, 91, 67, 3, 98, 4, 71, 62]], 'number of jobs': 20, 'number of machines': 10, 'initial seed': 587595453, 'upper bound and lower bound': 1582}


### Inicializar o relatorio para cada arquivo
relatório é uma lista de dicionarios

In [60]:
## traduzir para ingles
report_list = [
  {'solucao':[], 'aptidao': float('inf'), 'tempoFinal':0}
] * len(instances_list)
print(report_list)

[{'solucao': [], 'aptidao': inf, 'tempoFinal': 0}, {'solucao': [], 'aptidao': inf, 'tempoFinal': 0}, {'solucao': [], 'aptidao': inf, 'tempoFinal': 0}, {'solucao': [], 'aptidao': inf, 'tempoFinal': 0}, {'solucao': [], 'aptidao': inf, 'tempoFinal': 0}, {'solucao': [], 'aptidao': inf, 'tempoFinal': 0}, {'solucao': [], 'aptidao': inf, 'tempoFinal': 0}, {'solucao': [], 'aptidao': inf, 'tempoFinal': 0}, {'solucao': [], 'aptidao': inf, 'tempoFinal': 0}, {'solucao': [], 'aptidao': inf, 'tempoFinal': 0}]


Iniciar o laço de repetição para cada arquivo

## funções a serem implementadas


def makespan(instancia, solucao)
#copiar função fornecida

def lerInstancias(listaArquivos)
#ler a primeira instância de cada um dos dez arquivos, armazenar em uma lista e retornar essa lista

def criarPopulacaoInicial(instancia, tamanho)
#criar uma lista de soluções aleatórias (podem definir outro critério se desejarem) com o tamanho repassado

def avaliarPop(populacao)
#usar a função makespan para avalaiar a aptidão de cada elemento da população

def retornaMelhorSolucao(populacao, aptidaoPop)
#retorna a melhor solucao dentre a populacao atual

def selecionarPop(populacao, aptidaoPop)
#função deve retornar quais elementos serão recombinados e com quem (pode fazer uso da aptidao ou não para o critério de seleção)

def recombinacao(populacaoSelecionada)
#função deve usar o operador de recombinacao (definido pelo grupo_ para gerar novas soluções filhas

def mutacao(novasSolucoes)
#função deve usar o operador de mutacao (definido pelo grupo) para modificar as soluções filhas (não precisa ser todas)

def selecionarNovaGeracao(populacaoAtual, novasSolucoes)
#função deve criar uma nova população com as soluções novas (eliminando as antigas ou usando outro critério de seleção desejado)

def salvarRelatorio(relatorio)

In [55]:
for instance_index in range (len(instances_list)):
    
    # Começa vazio em todos os casos
    melhoresSolucoes = report_list[instance_index]

    # Para cada instância executar todo o algoritmo 10 vezes

    for i in range (10):
        current_instance = instances_list[i]
        best_solution = report_list[i]
        startTime = time.time()

        population = createPopulation(current_instance, POPULATION_SIZE)

        while True:
            if TIME_LIMIT <= time.time() - startTime:
                break
        ## criar segundo criterio de parada
            criterioParada2 = False
            if criterioParada2 == True:
                break

            aptidaoPop = avaliarPop(populacao)
            melhorSolucaoAtual = retornaMelhorSolucao(populacao, aptidaoPop)

            if melhorSolucao['aptidao'] > melhorSolucaoAtual['aptidao']:
                melhorSolucao = melhorSolucaoAtual
            
            populacaoSelecionada = selecionarPop(populacao, aptidaoPop)
            novasSolucoes = recombinacao(populacaoSelecionada)
            novasSolucoes = mutacao(novasSolucoes)
            populacao = selecionarNovaGeracao(populacao, novasSolucoes)
        pass
        
        melhorSolucao['tempoFinal'] = time.time() - startTime
        print(melhorSolucao)
        melhoresSolucoes = melhoresSolucoes | melhorSolucao
    report_list[instance_index] = melhoresSolucoes

  

a
a
a
a
a
a
a
a
a
a


Salvar relatório

In [ ]:
# salvarRelatorio(relatorio)
